In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(os.getcwd()+'/datasets/taxi_trip_data.csv')
df.head()

## Goal

The goal of this notebook is to clean and analize the data available for the purpose of later utilizing it in machine learning algorithms to generate predictions of fare amounts for potential rides and riders.

ML Goal: 

 -  plug in specific variables and generate an output that is as close as possible to what the actual fare turns out to be.

### Planning for Features
Here are the features of the current dataset that we'll keep, as well as a few that will need to be created based on other features, this is called feature engineering:

- pickup_timestamp
- dropoff_timestamp
- trip_distance
- fare_amount
- extra
- mta_tax
- imp_surcharge
- total_amount
- pickup_location_id
- dropoff_location_id
- 
### Exploration

One of the most powerful tools I have found for quickly calculating and visualizing the correlation between different values is a correlation matrix or heatmap.
The correlation matrix calculates how the change in one value effects a change in the other value, and assigns a value between -1 and 1 to that correlation.
Let's review what those correlation values mean before we move on:

-1 A very strong negative correlation, when value A moves in one direction, value B moves in the opposite direction.
0 No correlation between values A and B, when one moves, the other is not effected.
1 A very strong positive correlation, as you can guess, this is the opposite of the negative correlation above. When value A moves in one direction, value B follows in the same direction.

It's important to remember that this value isn't related to the rate of change, only the direction of change. Value A moves up, and value B either stays, moves up, or moves down.

Let's generate that matrix and plot it out in a heatmap!

In [ ]:
# Generating the correlation matrix
df_corr = df.select_dtypes(include=['number']).corr()

In [ ]:
# Drawing the heatmap
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(df_corr, cmap='YlGnBu', annot=True, linewidths=0.5);

So let's first list the values we should be focusing on here, we already have a list of known values that we should keeep, the only remaining values are:

vendor_id - Vendor of data provider. This definitely won't be used for anything for our model here

rate_code - The rate code at the end of the trip. Used likely to track certain charges. Has a correlation with tolls and tips but not much with anything else.

sotre_and_fwd_flag - This is simply a fag that indicates whether a value was stored in vehicle memory before being recorded due to a lack of internet connection. This is useless to us, however, it's currently stored as a string and converting it to a value that can appear in a correlation matrix later might serve useful. While not likely, it could be possible that values are different from those not stored in memory, such as having a higher amount of errors, or some upload process might be altering values in an unexpected way. I'll keep this for now and check for any correlations later.

Okay, so in the end, we're only dropping one column right off the start and that's vendor ID. We have a lot of data to work with so any small amount of change will make a larger than usual impact, so I'll drop that now and then talk about the next steps.

In [ ]:
df = df.drop('vendor_id', axis=1)
df.head()

### Cleaning

 - Remove duplicate rows - Carefully, as we only want to remove duplicate trips, not duplicates within the values themselves. These values are not required to be unique.
- Check for missing values
- Check for zeros and empty strings. These values won't be "missing" but still aren't valid. Very few columns in this data have valid zeros
- Validate formatting of data, especially dates
- Strip and normalize strings - our data doesn't contain any strings, so we can skip this.

In [ ]:
# Remove duplicates - Easy first step for reducing the size of the data, making following steps quicker  
# From here on, I'm going to rename the dataframe from df to td for temporary data, thus not altering the original dataframe until much later. 
td = df.drop_duplicates()
# less than 1% dropped
print(f"{df.shape[0] - td.shape[0]} duplicate rows dropped. Thats {df.shape[0] / td.shape[0] * 100}%")
print(f"{td.shape[0]} rows remain.")

In [ ]:
# Checking for missing values
for col in td.columns:
    missing = td[col].isna().sum()
    print(f"Missing values in {col}: {missing}")

# Checking for zeros in numeric columns
def check_for_zeros(td):
    for col in td.columns:
        zeros = td[td[col] == 0].shape[0]
        print(f"Zeros in {col}:{zeros}")
        
check_for_zeros(td)

Okay, first issue.
passenger_count, trip_distance, far_amount and total_amount all contain zeros. It doesn't appear to be a large amount of the overal data. I'd normally check what the percentage of the entire dataset these values account for, however, passenger_count has already been determined to not be kept and the rest of the information can't really be filled in with any mathematical methods because each of the values depends on the other to be calculated. Without distance, we can't determine fare amount, even with distance, it's impossible to know which miles were driven above the 12mph threshold and which were below. There isn't much of a choice but to drop these, however, total_amount can be corrected by simply adding all of the charge column values together, so I'll keep and fix these rows.

In [ ]:
# Dropping rows with 0 values in columns where 0 is not allowed
td = td.drop(['passenger_count'], axis=1)
td = td[td['trip_distance'] > 0]
td = td[td['fare_amount'] > 0]

check_for_zeros(td)

In [ ]:
# Checking how much of the original data ramains
remaining = td.shape[0] / df.shape[0] * 100
print(f"Remaining amount of original dataset: {remaining}%")

Data Formatting

In [ ]:
# Converting to an actual Python/Pandas datetime object ensures that the data is a valid datetime. Then we can move on to exploring the datetimes available.
td['pickup_datetime'] = pd.to_datetime(td['pickup_datetime'])
td['dropoff_datetime'] = pd.to_datetime(td['dropoff_datetime'])

print('Done.')

In [ ]:
td['year'] = pd.to_datetime(td['pickup_datetime']).dt.year
td['month'] = pd.to_datetime(td['pickup_datetime']).dt.month
td['day'] = pd.to_datetime(td['pickup_datetime']).dt.day
td['day_of_week'] = pd.to_datetime(td['pickup_datetime']).dt.dayofweek
td['hour_of_day'] = pd.to_datetime(td['pickup_datetime']).dt.hour

print('Done')

In [ ]:
# Converting the datetime columns to a numpy array for vectorization
pickup_array = td['pickup_datetime'].values
dropoff_array = td['dropoff_datetime'].values

# Getting the new timedelta, this takes less than a second to complete compared to 15+ minutes with apply()
trip_duration = np.subtract(dropoff_array, pickup_array)

# Adding the resulting array to the dataframe in the trip_duration column
td['trip_duration'] = pd.Series(trip_duration)

# Converting the timedelta to number of seconds
td['trip_duration'] = td['trip_duration'].dt.total_seconds()

# Preview the results
td.head()

In [ ]:
td.drop(['pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

In [ ]:
td.head()

In [ ]:
td = td[td['trip_duration'] > 0]

In [ ]:
list_of_years = td.year.unique()
print(list_of_years)

In [ ]:
for year in list_of_years:
    year_amount = td[td['year'] == year].shape[0]
    total_amount = td.shape[0]
    
    print(f"{year} makes up {(year_amount / total_amount) * 100}% of the dataset")

### Unexpected Results

It's clear that this dataset is VERY HEAVILY weighted towards 2018. For that reason, dropping anything from before 2018 can help avoid skewing the data towards old trends, while keeping anything newer than 2018 might reveal new trends, although, I don't think it will be super useful. If a dataset of this size consists of 99% of the same year, it's likely that the trips from newer years are either invalid data upon collection, and incomplete enough to actually show any trends.

In [ ]:
td = td[td['year'] == 2018]

td.describe()

In [ ]:
# Calculating total amounts and dropping rows whose values don't "add up"...
fare = td['fare_amount'].values
extra = np.add(fare, td['extra'].values)
mta_tax = np.add(extra, td['mta_tax'].values)
tip_amount = np.add(mta_tax, td['tip_amount'].values)
imp_surcharge = np.add(tip_amount, td['imp_surcharge'].values)
calculated_total_amount = np.add(imp_surcharge, td['tolls_amount'].values)

td['calculated_total_amount'] = pd.Series(calculated_total_amount)

# validate calculated total by manually adding all relevant columns and comparing to the calculated column
td.head(10)

In [ ]:
# Dropping anything that isn't correct
td = td[td['total_amount'] != td['calculated_total_amount']]

td.head()


### Finishing Up
That one total_amount column did a lot more than just clean totals, but it actually checked all of the other total effecting columns at the same time. If any errors occurred in any column, the calculated total would have differed from the calculated total. Missing mta_tax? Dropped. Incorrect amount of tolls? Dropped.

That about sums up the cleaning process. While there are still some values that need to be looked further into, such as storoe_and_fwd_flag, anything from this point forward will rely heavily on the exploration phase to understand and determine what to do with. For now, I'll save the cleaned data as a csv.

In [ ]:
td.to_csv('cleaned_nyc_taxi_data_2018.csv')
print('Done!')

## Excersise

 - Basic Exploration → Missing values, summary stats.

 - Data Cleaning → Convert datatypes, handle unrealistic values.

 - NumPy Operations → Moving averages, per-mile calculations.

 - Pandas Grouping → Aggregating trip counts and fares.

 - Advanced Analytics → Outlier detection, busiest locations.

 - Data Visualization → Line plots, heatmaps.

 - Bonus (ML) → Train a basic fare prediction model.

Identify the top 5 busiest pickup locations and visualize them

In [ ]:
top_pickup_locations = td['pickup_location_id'].value_counts().head(5)

# Visualization
plt.figure(figsize=(8, 5))
sns.barplot(x=top_pickup_locations.index, y=top_pickup_locations.values, palette="viridis")
plt.xlabel("Pickup Location ID")
plt.ylabel("Number of Pickups")
plt.title("Top 5 Busiest Pickup Locations")
plt.show()

In [ ]:
# Detect outliers in trip duration using box plots and standard deviation
#Data Cleaning & Preprocessing

#Filter out unrealistic trip durations (e.g., < 1 min or > 24 hours):


In [2]:
td = td[(td['trip_duration'] > 1) & (td['trip_duration'] < 1440)]

NameError: name 'td' is not defined

NumPy-Based Exercises


In [3]:
#Compute the average fare per mile using NumPy

In [ ]:
import numpy as np
fares_per_mile = np.where(trips['trip_distance'] > 0, trips['fare_amount'] / trips['trip_distance'], np.nan)
print(np.nanmean(fares_per_mile))  # Average fare per mile

In [ ]:
#Compute a moving average of fares over time:

In [ ]:
def moving_avg(arr, window):
    return np.convolve(arr, np.ones(window)/window, mode='valid')

avg_fares = moving_avg(trips['fare_amount'].dropna().values, 5)
print(avg_fares[:10])  # First 10 values


Data Aggregation & Grouping

In [ ]:
#Find average fare by hour of the day:

In [ ]:
trips['hour'] = trips['pickup_datetime'].dt.hour
avg_fare_per_hour = trips.groupby('hour')['fare_amount'].mean()
print(avg_fare_per_hour)


In [ ]:
#Find total trips per day of the week:

In [ ]:
trips['day_of_week'] = trips['pickup_datetime'].dt.dayofweek
trips_per_day = trips.groupby('day_of_week')['pickup_datetime'].count()
print(trips_per_day)


Advanced Analytics

In [4]:
#Find the busiest pickup locations over time:


In [ ]:
busiest_hours = trips.groupby(['pickup_location_id', 'hour']).size().reset_index(name='count')
busiest_hours = busiest_hours.sort_values(by='count', ascending=False).head(10)
print(busiest_hours)

In [5]:
#Detect outliers in fare amounts using Z-score:



In [ ]:
from scipy import stats
trips['fare_zscore'] = stats.zscore(trips['fare_amount'])
outliers = trips[trips['fare_zscore'].abs() > 3]  # 3 standard deviations away
print(outliers)


Data Visualization

In [6]:
#Plot fare trends by time of day using Seaborn:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
sns.lineplot(x=avg_fare_per_hour.index, y=avg_fare_per_hour.values, marker='o')
plt.xlabel("Hour of Day")
plt.ylabel("Average Fare")
plt.title("Average Taxi Fare by Hour of the Day")
plt.grid()
plt.show()

In [7]:
#Plot a heatmap of correlations between numerical variables

In [ ]:
corr_matrix = trips[['fare_amount', 'trip_distance', 'trip_duration']].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.show()

In [8]:
#Machine Learning 

In [9]:
#Build a simple fare prediction model using Linear Regression:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

features = trips[['trip_distance', 'passenger_count']]
target = trips['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(predictions[:10])  # First 10 predictions
